# TMA4215 - Assignment 2 - Conjugate Gradient and ODEs

**Deadline:** Wednesday September 23, 11:59PM.


In this assignment you will implement the Preconditioned Conjugate Gradient (PCG) method.
Then there are some small theoretical questions on Numerical ODEs.

**PCG algorithm.** Implementations of the (P)CG algorithm can be found in many libraries, but here you are supposed to implement your own code for solving $Ax=b$ by PCG. Here we shall assume that the matrix $A$ is implemented by means of a function, say *A_func(v)* that for any vector $v$ returns $Av$. Similarly, we implement the preconditioner also as a function *Precond(v)* that returns $P^{-1}v$ for any vector $v$. An popular alternative to this approach is to use sparse representation of matrices, (see e.g. *scipy.sparse*), but we shall not use that here.

You are of course allowed to use the obvious built-in functions for elementary operations, such as computing inner products, norms etc. We will provide the function *A_func(v)* to be used in this assignment. Your job is to implement a function *PCG* whose template you find below, a function *Precond(v)* that implements some iterations of the Gauss-Jacobi splitting method as preconditioner, and finally a small main programme that generates the results.

**Problem 1.** 

**(a)** Fill in the templates below for a function *PCG* 

*PCG* should take 5 input arguments, 
1. the function *A_func* that takes $v$ as input and returns $A\cdot v$.
2. the right hand side *b* of the system $A\cdot x = b$ (numpy array)
3. The initial guess *x0*
4. *tol*, a tolerance used to define the stopping criterion
5. *Precond* which is a function that takes a vector $v$ as input and returns $P^{-1}v$ for the preconditioner $P$.

There should be 3 output arguments: 
1. the computed solution $x$, 
2. the residual after the last iteration, $r$ 
3. the number of iterations *iter* needed to meet the stopping criterion 

The stopping criterion can be taken as ($\|r\|<$*tol* or *iter*$\geq N$) where $N$ is the dimension of the problem.



**(b)** Write a Gauss-Jacobi preconditioner function (see below *PrecJac*)
Test your implementation using the supplied function *A_func* which takes as input a vector $v\in\mathbb{R}^{n^2}$ and returns the vector in $A\cdot v\in\mathbb{R}^{n^2}$. This corresponds to a discretisation of the Poisson equation

$$ -\nabla^2 u = f $$

in the plane, so that there are $n$ discretisation points in each direction, thereby dimension $n^2$ of the unknown solution vector. You can take *x0* to be the 0-vector in $\mathbb{R}^{n^2}$. The right hand side is also obtained by a supplied function *b_def* (see below) that from an input integer $n$ returns a vector $b\in\mathbb{R}^{n^2}$.

*PrecJac(v)*. This function can be considered as tailored for the example given by *A_func*.
Starting with the 0-vector as initial guess, apply a specified number of iterations with the Gauss-Jacobi iterative method to the problem $Au=v$. The number of iterations can be specified in the main program. 
You need the diagonal of $A$ to construct the preconditioner and you can assume that $M=\text{diag}(A)=4I$ (four times the identity matrix). Therefore, via $A=M-N$ in Gauss-Jacobi, you can write $N=M-A=4I-A$ so that the GJ-iteration becomes

$$
     u^{(k+1)} = M^{-1}N u^{k} + M^{-1}b = (I-\frac14 A)u^{(k)}+\frac 14 b = u^{(k)}+\frac14 (b-Au^{(k)}),\quad
     u^{(0)} = 0
$$

Try to run the program with and without preconditioner for $N=n^2, \ n=20, 40, 60, \ldots, 200$. For each $n$, print out a line with $n$, *iterNull* and *iterJac2*, and *iterJac4*. Here *iterNull* is without preconditoning,
*iterJac2* and *iterJac4* is with 2 and 4 Gauss-Jacobi iterations respectively.

You can use *tol*$=10^{-5}$ in all experiments. Always let *x0* be the zero-vector.

**Control question 1.** How many CG iterations are needed with no preconditioning, $N=n^2=256 (n=16)$ with *tol* and *x0* as specified above.

**Control question 2.** How many PCG iterations are needed with preconditioning, 4 Gauss-Jacobi iterations, $N=n^2=256 (n=16)$ with *tol* and *x0* as specified above.


In [29]:
import numpy as np
import numpy.linalg as la

def PCG(A_func,b,x0,tol,Precond):
    iter = 0
    x = x0
    r = b - A_func(x0)
    z = Precond(r)
    p = z
    while (np.linalg.norm(r) > tol):
        alpha = np.inner(r, z) / np.inner(p, A_func(p))
        x = x + alpha * p
        r = r - alpha * A_func(p)
        
        z_new = Precond(r)
        beta = np.inner(A_func(p), z_new) / np.inner(A_func(p), p)
        p = z_new - beta * p
        
        z = z_new
        iter = iter + 1

    return x, iter, r
    
    
def PrecNull(r):
    return r

def PrecJac(v):
    u = np.zeros(len(v))
    b = b_def(np.int(np.sqrt(len(v))))[1]
    global iterations
    for i in range(iterations):
        u = u + 1/4 * (b - A_func(u))
        
    return u # which equals P^{-1}v


In [30]:
# Supplied functions you can use for Problem 1b.
import sys

def A_func(v):
# Note that v of length N must be square, N=n**2 for some n.
    n2=len(v)
    n=int(np.sqrt(n2))
    if n**2 != n2:
        sys.exit("Size of vector must be a square")
    M=np.zeros((n+2,n+2))
    M[1:n+1,1:n+1]=np.reshape(v,(n,n))
    AM = 4*M[1:n+1,1:n+1]-M[:n,1:n+1]-M[2:n+2,1:n+1]-M[1:n+1,:n]-M[1:n+1,2:n+2]
    Av = np.reshape(AM,(n2,))
    return Av

def b_def(n):
    h = 1/(n+1)
    X = np.linspace(h,1-h,n)
    Y=X
    B = np.outer( 16*X**2*(1-X)**2, 16*Y**2*(1-Y)**2 )
    b = np.reshape(B, (n**2,))
    return B,b


In [31]:
if __name__ == '__main__':
    tol = 1e-5
    global iterations

    for n in np.arange(20,220,20):
        x0 = np.zeros(n*n)
        b = b_def(n)[1]
        
        # No preconditioning
        x, iterNull, r = PCG(A_func,b,x0,tol,PrecNull)
        
        # Two iterations on preconditioning
        iterations = 2
        x, iterJac2, r = PCG(A_func,b,x0,tol,PrecJac)
        
        # Four iterations on preconditioning
        iterations = 4
        x, iterJac4, r = PCG(A_func,b,x0,tol,PrecJac)
        
        print("n:", n, "\t iterNull:", iterNull, "\t iterJac2:", iterJac2, "\t iterJac4:", iterJac4)
        
        

/home/anderstf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/home/anderstf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':
/home/anderstf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in multiply
  del sys.path[0]
/home/anderstf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


n: 20 	 iterNull: 26 	 iterJac2: 2 	 iterJac4: 2
n: 40 	 iterNull: 53 	 iterJac2: 2 	 iterJac4: 2
n: 60 	 iterNull: 81 	 iterJac2: 2 	 iterJac4: 2
n: 80 	 iterNull: 109 	 iterJac2: 2 	 iterJac4: 2
n: 100 	 iterNull: 137 	 iterJac2: 2 	 iterJac4: 2
n: 120 	 iterNull: 168 	 iterJac2: 2 	 iterJac4: 2
n: 140 	 iterNull: 198 	 iterJac2: 2 	 iterJac4: 2
n: 160 	 iterNull: 226 	 iterJac2: 2 	 iterJac4: 2
n: 180 	 iterNull: 255 	 iterJac2: 2 	 iterJac4: 2
n: 200 	 iterNull: 283 	 iterJac2: 2 	 iterJac4: 2


**Problem 2.** 

**(a)** Find the stability function of the Runge-Kutta method whose Butcher tableaux is given as

$$
\begin{array}{c|rr}
\frac12 & \frac12 & 0 \\
\frac32 & -\frac12 & 2 \\ \hline
        & -\frac12 & \frac32
\end{array}
$$

**(b)** Does the stability region of this method contain the left half plane, $\mathbb{C}^-=\{z\in\mathbb{C}: \text{Re}(z)<0\}$?

**Control question 3.** A multiple choice question for the stability function.

**Control question 4.** A yes/no answer for **(2b)**



**Answer**

**a)** Writing the stability function as

$$
    R(z) = \frac{P(z)}{Q(z)},
$$
where

$$
\begin{align*}
    P(z) &= det(I - zA + z \mathbf{1} b^\top), \\
    Q(z) &= det(I - zA),
\end{align*}
$$

with $A$ and $b$ given by the Butcher tableaux above, we get the stability function

$$
\begin{align*}
    R(z) &= \frac{(1-z)(1-z/2)}{(1-z/2)(1-2z)} \\
    &= \frac{1-z}{1-2z} \\
    &= \frac{z-1}{2z-1}.
\end{align*}
$$

**b)** The region of stability is given as

$$
    S_R = \{z \in \mathbb{C} : |R(z)| < 1 \}.
$$

As $|2z-1| > |z-1|$ for all $Re(z) < 0$, we have that the stability region contains the left half-plane.